In [2]:
import wot
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy

In [3]:
# We only need moscot here to calculate the initial growth rates
from moscot.problems.time._lineage import TemporalProblem

/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'interpolation_parameters' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/docrep/decorators.py:43: SyntaxWarning: 'account_for_unbalancedness' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/home/mg

In [4]:
def compute_initial_growth_rates(adata):
    # Get prior growth rates
    tp=TemporalProblem(adata)
    tp=tp.score_genes_for_marginals(gene_set_proliferation='mouse',  gene_set_apoptosis='mouse')
    tp = tp.prepare('day', joint_attr=f'X_pcaS')
    gr=tp.prior_growth_rates.loc[adata.obs.index]['prior_growth_rates'].values
    return(gr)

In [5]:
Path="/home/mgander/mouse_atlas/data"
ts=['E3.5', 'E4.5', 'E5.25', 'E5.5', 'E6.25', 'E6.5', 'E6.75', 'E7.0', 'E7.25', 'E7.5', 'E7.75', 'E8.0', 'E8.25', 'E8.5a', 'E8.5b', 'E9.5', 'E10.5', 'E11.5', 'E12.5', 'E13.5']

# This computes the WOT maps

In [ ]:
for i in range(13):
    ts0=ts[i]
    ts1=ts[i+1]
    print(f'{ts0}_{ts1}')

    adata=sc.read(f"{Path}/Comb_anndatas/adata_{ts0}_{ts1}.h5ad")
    del adata.raw
    # Compute intiial growth rates the same as in moscot
    adata.obs['cell_growth_rate']=compute_initial_growth_rates(adata)
    adata=anndata.AnnData(X=adata.obsm['X_pcaS'], obs=adata.obs)

    day0,day1=sorted(set(adata.obs['day']))

    inds0=list(adata[adata.obs['day']==day0].obs.index)
    inds1=list(adata[adata.obs['day']==day1].obs.index)


    # Parameter choice the same as in moscot
    eps=0.005
    tau1s=[0.99,0.98,0.99,0.92,0.95,0.9,0.3,0.65,0.9,0.82,0.93,0.97,0.96]
    lam1s=[t*eps/(1-t) for t in tau1s]
    lam2=100
    lam1=lam1s[i]

    # Compute optimal transport map
    ot_model = wot.ot.OTModel(adata, epsilon = eps, lambda1 = lam1, lambda2 = lam2, growth_iters=1) 
    tmap = ot_model.compute_transport_map(day0, day1)

    # Restructure it into a DataFrame and save it
    T=pd.DataFrame(data=tmap.X, index=inds0, columns=inds1)
    T=T/T.sum()
    T.to_pickle(f'{Path}/WOT_maps/Ts/Transitions_{ts0}_{ts1}.pkl')

E3.5_E4.5
Only considering the two last: ['.5', '.h5ad'].
Only considering the two last: ['.5', '.h5ad'].


/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/wot/ot/optimal_transport.py:160: RuntimeWarning: invalid value encountered in double_scalars
  np.linalg.norm(_b - old_b * np.exp(v / epsilon_i)) / (1 + np.linalg.norm(_b)))
/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/wot/ot/optimal_transport.py:148: RuntimeWarning: overflow encountered in exp
  _a = a * np.exp(u / epsilon_i)
/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/wot/ot/optimal_transport.py:149: RuntimeWarning: overflow encountered in exp
  _b = b * np.exp(v / epsilon_i)


E4.5_E5.25
Only considering the two last: ['.25', '.h5ad'].
Only considering the two last: ['.25', '.h5ad'].


/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/wot/ot/optimal_transport.py:160: RuntimeWarning: invalid value encountered in double_scalars
  np.linalg.norm(_b - old_b * np.exp(v / epsilon_i)) / (1 + np.linalg.norm(_b)))
/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/wot/ot/optimal_transport.py:149: RuntimeWarning: overflow encountered in exp
  _b = b * np.exp(v / epsilon_i)


E5.25_E5.5
Only considering the two last: ['.5', '.h5ad'].
Only considering the two last: ['.5', '.h5ad'].
E5.5_E6.25
Only considering the two last: ['.25', '.h5ad'].
Only considering the two last: ['.25', '.h5ad'].
E6.25_E6.5
Only considering the two last: ['.5', '.h5ad'].
Only considering the two last: ['.5', '.h5ad'].
E6.5_E6.75
Only considering the two last: ['.75', '.h5ad'].
Only considering the two last: ['.75', '.h5ad'].
E6.75_E7.0
Only considering the two last: ['.0', '.h5ad'].
Only considering the two last: ['.0', '.h5ad'].


/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/wot/ot/optimal_transport.py:149: RuntimeWarning: overflow encountered in exp
  _b = b * np.exp(v / epsilon_i)


E7.0_E7.25
Only considering the two last: ['.25', '.h5ad'].
Only considering the two last: ['.25', '.h5ad'].
